# DRYML Tutorial

## Ray tune

One primary use case for DRYML is hyperparameter tuning of your models. `ObjectDef` provides an easy way to define and construct models. The `train` method of trainables allows uniform training for all models, and DRYML provides a set of interface methods to easily run hyperparameters searches on your models.

We'll briefly introduce `ray`, and write a simple hyperparameter tuning example.

RAY is a platform for remote process execution. It creates a server which manages connected resources. Jobs can then be sent to those resources in the form of multiple processes confined to specific resources. This is ideal for hyperparameter tuning, and in fact RAY provides the `ray.tune` library for exactly this.

This tutorial won't serve as a tutorial for ray, for that please consult the ray documentation available here: https://docs.ray.io/en/latest/index.html and here: https://docs.ray.io/en/latest/tune/index.html

Let's start the ray server, and write a simple method for generating models.

In [1]:
import ray

In [2]:
ray.init(num_gpus=1, num_cpus=8)

RayContext(dashboard_url='', python_version='3.8.13', ray_version='1.12.1', ray_commit='{{RAY_COMMIT_SHA}}', address_info={'node_ip_address': '141.142.145.46', 'raylet_ip_address': '141.142.145.46', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-10-12_11-56-03_985207_1586169/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-10-12_11-56-03_985207_1586169/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-10-12_11-56-03_985207_1586169', 'metrics_export_port': 65172, 'gcs_address': '141.142.145.46:51310', 'address': '141.142.145.46:51310', 'node_id': '1e265ad32bde52cd1f13a870071b5eb38d2d6f8168f730a9e18fc87a'})

## DRYML support

DRYML provides support for `ray.tune` in the form of the `dryml.ray.tune.Trainer` class. This class defines a callable function compatible with the ray tune functional API. We just need to supply it with a special callable which can provide a few needed callable methods to setup and run the tune experiment.

`dryml.ray.tune.Trainer` expects the arguments:
* `name`: The name of the experiment to use
* `prep_method`: The callable for creating the necessary callables for setting up the experiment. This must be picklable via `dill`.
* `metrics`: A dictionary of metrics to compute after each step of training.

Once created, the user can then design their tune experiment in the usual way, and pass the `Trainer` as the callable trainable method.

We'll create a `prep_method` which can yield all needed callables for a simple experiment: How large a convolutional kernel is appropriate for a two layer convolutional model for classifying MNIST digits.

## Define `prep_method`

In [3]:
import dryml
import os

In [4]:
# Name experiment so we can set model directory
experiment_name = 'TF_ray_test'
model_dir = os.path.realpath(os.path.join('./models', experiment_name))

# A callable to create the train/test `Dataset`
def data_gen():
    import tensorflow_datasets as tfds
    from dryml.data.tf import TFDataset
    
    # Check whether tensorflow support exists
    # For the current GPU.
    dryml.context.context_check({'tf': {}})
    
    (ds_train, ds_test), ds_info = tfds.load(
        'mnist',
        split=['train', 'test'],
        shuffle_files=True,
        as_supervised=True,
        with_info=True)
    
    train_ds = TFDataset(
        ds_train,
        supervised=True
    )
    test_ds = TFDataset(
        ds_test,
        supervised=True
    )
    return {
        'train': train_ds,
        'test': test_ds,
    }


def prep_method():
    # A callable to create a repo. This is needed to store completed models for later use.
    def repo_gen():
        return dryml.Repo(directory=model_dir, create=True)


    # We need another callable since the input datasets have a context
    # Requirement, we want the Trainer function to incorporate this
    # requirement when building the compute context.
    def data_ctx_gen():
        return {'tf': {}}

    # Model generator method which takes a config, and generates a model
    # It also can take a repo keyword argument so already trained components
    # Can be grabbed from the repo.
    def model_gen(config, repo=None):
        import dryml
        import dryml.models
        import dryml.models.tf
        import tensorflow as tf
        
        # Grab the existing Best Category data transformation
        best_cat_def = dryml.ObjectDef(dryml.data.transforms.BestCat)
        best_step = repo.get(best_cat_def)
        
        kernel_size = int(config['kernel_size'])

        filters = 32
        n_layers = 2
        layer_defs = []
        for i in range(n_layers):
            layer_defs.append(
                ['Conv2D', {'filters': filters, 'kernel_size': kernel_size, 'activation': 'relu'}])
        layer_defs.append(['Flatten', {}])
        layer_defs.append(['Dense', {'units': 10, 'activation': 'linear'}])
        
        mdl_def = dryml.ObjectDef(
            dryml.models.tf.keras.base.SequentialFunctionalModel,
            input_shape=(28, 28, 1),
            layer_defs=layer_defs,
        )
        
        # Create model definition
        mdl_def = dryml.ObjectDef(
            dryml.models.Pipe,
            dryml.ObjectDef(
                dryml.models.tf.keras.Trainable,
                # Model definition
                model=mdl_def,
                # Train method
                train_fn=dryml.ObjectDef(
                    dryml.models.tf.keras.base.BasicTraining,
                    epochs=5,
                ),
                # Optimizer
                optimizer=dryml.ObjectDef(
                    dryml.models.tf.ObjectWrapper,
                    tf.keras.optimizers.Adam,
                ),
                # Loss
                loss=dryml.ObjectDef(
                    dryml.models.tf.ObjectWrapper,
                    tf.keras.losses.SparseCategoricalCrossentropy,
                    obj_kwargs={
                        'from_logits': True
                    }
                )
            ),
            best_step,
        )
        
        ctx_reqs = {'tf': {'num_gpus': 1}}

        # Return dictionary with the model and optionally, 
        return {
            'model': mdl_def.build(repo=repo),
            'ctx_reqs': ctx_reqs,
        }

    # Return dictionary with defined callables.
    return {
        'repo': repo_gen,
        'data_ctx': data_ctx_gen,
        'data': data_gen,
        'model': model_gen,
    }

Next, let's prepare the `Repo` with needed objects and the `Repo` directory.

In [5]:
# Let's create a repo pointing to the same directory.
repo = dryml.Repo(directory=model_dir, create=True)

# Create a Best Category trainable, and save it to the repo.
best_cat_def = dryml.ObjectDef(dryml.data.transforms.BestCat)
# Repo's get method has a special ability when `build_missing_def=True`.
# If a non-concrete definition is not in the repo, one instance will be
# created and stored in the repo.
repo.get(best_cat_def, build_missing_def=True)

# Save the objects
repo.save()

Now, let's design the tune experiment!

In [ ]:
from ray import tune
import dryml.ray
import dryml.metrics
import datetime

local_dir = os.path.realpath(f'./ray_results')

# Define study space
config = {
    'kernel_size': tune.randint(3,8),
}

# Create model trainer
model_trainer = dryml.ray.tune.Tune1Trainer(
    name=experiment_name,
    prep_method=prep_method,
    metrics={'accuracy': dryml.metrics.scalar.categorical_accuracy},
)

analysis = tune.run(
    model_trainer,
    config=config,
    metric='accuracy',
    mode='max',
    resources_per_trial={'cpu': 8, 'gpu': 1},
    num_samples=20,
    keep_checkpoints_num=2,
    local_dir=local_dir,
    log_to_file=True,
    reuse_actors=False,
    fail_fast=True,
    progress_reporter=tune.JupyterNotebookReporter(
        False,
        max_report_frequency=30,
    ),
    resume='AUTO',
)

2022-10-12 11:56:08,323	INFO registry.py:79 -- Detected unknown callable for trainable. Converting to class.
/projects/bbkg/mkrafcz2/dryml/dryml/dryml_venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-10-12 11:56:08,899	INFO trial_runner.py:508 -- No local checkpoint was found. Ray Tune will now start a new experiment.
2022-10-12 11:56:09,036	INFO trial_runner.py:803 -- starting TF_ray_test_c4844_00000


Trial name,status,loc,kernel_size
TF_ray_test_c4844_00000,RUNNING,141.142.145.46:1586581,3
TF_ray_test_c4844_00001,PENDING,,7
TF_ray_test_c4844_00002,PENDING,,3
TF_ray_test_c4844_00003,PENDING,,7
TF_ray_test_c4844_00004,PENDING,,4
TF_ray_test_c4844_00005,PENDING,,4
TF_ray_test_c4844_00006,PENDING,,6
TF_ray_test_c4844_00007,PENDING,,6
TF_ray_test_c4844_00008,PENDING,,7
TF_ray_test_c4844_00009,PENDING,,6


(func pid=1586581) 2022-10-12 11:56:11.664690: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
(func pid=1586581) 2022-10-12 11:56:13.326981: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
(func pid=1586581) 2022-10-12 11:56:13.327534: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
(func pid=1586581) 2022-10-12 11:56:13.627956: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
(func pid=1586581) pciBusID: 0000:07:00.0 name: NVIDIA A100-SXM4-40GB computeCapability: 8.0
(func pid=1586581) coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.45GiB deviceMemoryBandwidth: 1.41TiB/s
(func pid=1586581) 2022-10-12 11:56:13.627993: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
(func pid=15

Trial name,status,loc,kernel_size
TF_ray_test_c4844_00000,RUNNING,141.142.145.46:1586581,3
TF_ray_test_c4844_00001,PENDING,,7
TF_ray_test_c4844_00002,PENDING,,3
TF_ray_test_c4844_00003,PENDING,,7
TF_ray_test_c4844_00004,PENDING,,4
TF_ray_test_c4844_00005,PENDING,,4
TF_ray_test_c4844_00006,PENDING,,6
TF_ray_test_c4844_00007,PENDING,,6
TF_ray_test_c4844_00008,PENDING,,7
TF_ray_test_c4844_00009,PENDING,,6


Trial name,status,loc,kernel_size
TF_ray_test_c4844_00000,RUNNING,141.142.145.46:1586581,3
TF_ray_test_c4844_00001,PENDING,,7
TF_ray_test_c4844_00002,PENDING,,3
TF_ray_test_c4844_00003,PENDING,,7
TF_ray_test_c4844_00004,PENDING,,4
TF_ray_test_c4844_00005,PENDING,,4
TF_ray_test_c4844_00006,PENDING,,6
TF_ray_test_c4844_00007,PENDING,,6
TF_ray_test_c4844_00008,PENDING,,7
TF_ray_test_c4844_00009,PENDING,,6


...

Trial name,status,loc,kernel_size,iter,total time (s),accuracy
TF_ray_test_c4844_00012,RUNNING,141.142.145.46:1608989,3,,,
TF_ray_test_c4844_00013,PENDING,,5,,,
TF_ray_test_c4844_00014,PENDING,,6,,,
TF_ray_test_c4844_00015,PENDING,,6,,,
TF_ray_test_c4844_00016,PENDING,,6,,,
TF_ray_test_c4844_00017,PENDING,,7,,,
TF_ray_test_c4844_00018,PENDING,,3,,,
TF_ray_test_c4844_00019,PENDING,,6,,,
TF_ray_test_c4844_00000,TERMINATED,141.142.145.46:1586581,3,6,485.43,0.0979567
TF_ray_test_c4844_00001,TERMINATED,141.142.145.46:1588848,7,6,539.3,0.0979567


Trial name,status,loc,kernel_size,iter,total time (s),accuracy
TF_ray_test_c4844_00012,RUNNING,141.142.145.46:1608989,3,,,
TF_ray_test_c4844_00013,PENDING,,5,,,
TF_ray_test_c4844_00014,PENDING,,6,,,
TF_ray_test_c4844_00015,PENDING,,6,,,
TF_ray_test_c4844_00016,PENDING,,6,,,
TF_ray_test_c4844_00017,PENDING,,7,,,
TF_ray_test_c4844_00018,PENDING,,3,,,
TF_ray_test_c4844_00019,PENDING,,6,,,
TF_ray_test_c4844_00000,TERMINATED,141.142.145.46:1586581,3,6,485.43,0.0979567
TF_ray_test_c4844_00001,TERMINATED,141.142.145.46:1588848,7,6,539.3,0.0979567


Trial name,status,loc,kernel_size,iter,total time (s),accuracy
TF_ray_test_c4844_00012,RUNNING,141.142.145.46:1608989,3,,,
TF_ray_test_c4844_00013,PENDING,,5,,,
TF_ray_test_c4844_00014,PENDING,,6,,,
TF_ray_test_c4844_00015,PENDING,,6,,,
TF_ray_test_c4844_00016,PENDING,,6,,,
TF_ray_test_c4844_00017,PENDING,,7,,,
TF_ray_test_c4844_00018,PENDING,,3,,,
TF_ray_test_c4844_00019,PENDING,,6,,,
TF_ray_test_c4844_00000,TERMINATED,141.142.145.46:1586581,3,6,485.43,0.0979567
TF_ray_test_c4844_00001,TERMINATED,141.142.145.46:1588848,7,6,539.3,0.0979567


Trial name,status,loc,kernel_size,iter,total time (s),accuracy
TF_ray_test_c4844_00012,RUNNING,141.142.145.46:1608989,3,,,
TF_ray_test_c4844_00013,PENDING,,5,,,
TF_ray_test_c4844_00014,PENDING,,6,,,
TF_ray_test_c4844_00015,PENDING,,6,,,
TF_ray_test_c4844_00016,PENDING,,6,,,
TF_ray_test_c4844_00017,PENDING,,7,,,
TF_ray_test_c4844_00018,PENDING,,3,,,
TF_ray_test_c4844_00019,PENDING,,6,,,
TF_ray_test_c4844_00000,TERMINATED,141.142.145.46:1586581,3,6,485.43,0.0979567
TF_ray_test_c4844_00001,TERMINATED,141.142.145.46:1588848,7,6,539.3,0.0979567


2022-10-12 13:35:24,749	WARNING tune.py:650 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


In [45]:
# Finally, we can inspect the results of this experiment and have a look at the best model!
best_result = results.get_best_result(metric='accuracy', mode='max')
print(f"Best configuration: {best_result.config}")
print(f"Accuracy: {best_result.metrics['accuracy']}")
best_model_id = best_result.metrics['dry_id']
print(f"id: {best_model_id}")

Best configuration: {'kernel_size': 4}
Accuracy: 0.9782652243589743
id: ce6ba022-cbad-464c-b308-815d80d9a936


In [46]:
# Refresh the repository
repo.load_objects_from_directory()

In [47]:
# Fetch the best performing model
model = repo.get_obj_by_id(best_model_id)

In [55]:
# Lets define a method to test the model's accuracy
@dryml.compute
def test_model(model):
    import dryml.metrics
    import tensorflow_datasets as tfds
    from dryml.data.tf import TFDataset

    # Check whether tensorflow support exists
    # For the current GPU.
    dryml.context.context_check({'tf': {}})

    (ds_test,) = tfds.load(
        'mnist',
        split=['test'],
        shuffle_files=True,
        as_supervised=True)

    test_ds = TFDataset(
        ds_test,
        supervised=True
    )

    return dryml.metrics.scalar.categorical_accuracy(model, test_ds)

In [56]:
# And verify recorded accuracy!
test_model(model)

0.9782652243589743